In [36]:
import pandas as pd
import numpy as np

In [64]:

# Read the CSV file
df = pd.read_csv('Books.csv')

# Function to update URL from HTTP to HTTPS
def update_url(url):
    if url.startswith('http://'):
        return 'https://' + url[len('http://'):]
    return url

# Update URLs in the DataFrame
df['Image-URL-M'] = df['Image-URL-M'].apply(update_url)

# Write back to CSV
df.to_csv('updated_books.csv', index=False)


C:\Users\Afshan Perween\AppData\Local\Temp\ipykernel_6608\1688409960.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Books.csv')


In [65]:
books = pd.read_csv('updated_books.csv')


C:\Users\Afshan Perween\AppData\Local\Temp\ipykernel_6608\912418304.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('updated_books.csv')


In [66]:
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

In [67]:
books['Image-URL-M'][1]

'https://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg'

In [68]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [69]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [70]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [71]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [72]:
books.duplicated().sum()

0

In [73]:
ratings.duplicated().sum()

0

In [74]:
users.duplicated().sum()

0

In [75]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [76]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
users_goodrating= x[x].index

In [77]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(users_goodrating)]

In [78]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [79]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [80]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [81]:
pt.fillna(0,inplace=True)

In [82]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
from sklearn.metrics.pairwise import cosine_similarity

In [92]:
similarity_scores = cosine_similarity(pt)

In [93]:
similarity_scores.shape

(706, 706)

In [86]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [87]:
recommend('Animal Farm')

[['1984',
  'George Orwell',
  'https://images.amazon.com/images/P/0451524934.01.MZZZZZZZ.jpg'],
 ['Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson',
  'Louise Rennison',
  'https://images.amazon.com/images/P/0064472272.01.MZZZZZZZ.jpg'],
 ['Midnight',
  'Dean R. Koontz',
  'https://images.amazon.com/images/P/0425118703.01.MZZZZZZZ.jpg'],
 ['Second Nature',
  'Alice Hoffman',
  'https://images.amazon.com/images/P/0399139087.01.MZZZZZZZ.jpg']]

In [88]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,https://images.amazon.com/images/P/0195153448....,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,https://images.amazon.com/images/P/0002005018....,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,https://images.amazon.com/images/P/0060973129....,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,https://images.amazon.com/images/P/0374157065....,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,https://images.amazon.com/images/P/0393045218....,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,https://images.amazon.com/images/P/0449906736....,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,https://images.amazon.com/images/P/0525447644....,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,https://images.amazon.com/images/P/006008667X....,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,https://images.amazon.com/images/P/0192126040....,http://images.amazon.com/images/P/0192126040.0...


In [51]:
pt.index[584]

'The Phantom Tollbooth'

In [89]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import ndcg_score

# Assuming similarity_scores and pt are already calculated
# Split data into train and test sets (you can use any method, e.g., train_test_split)
train_data = final_ratings.sample(frac=0.8, random_state=42)
test_data = final_ratings.drop(train_data.index)

# Train the recommendation model (not applicable here as it's similarity-based)
# If you have a model that needs training, do it here

# Generate recommendations for each user in the test set
all_recommendations = {}
for user_id in test_data['User-ID'].unique():
    recommendations = recommend(user_id)  # Assuming you have a function to generate recommendations for a user
    all_recommendations[user_id] = recommendations

# Prepare ground truth data for users in the test set
ground_truth = {}
for user_id in test_data['User-ID'].unique():
    ground_truth[user_id] = final_ratings[(final_ratings['User-ID'] == user_id) & (final_ratings['Book-Rating'] >= 7)]['Book-Title'].tolist()

# Calculate evaluation metrics
precision = precision_score(ground_truth, all_recommendations, average='micro')
recall = recall_score(ground_truth, all_recommendations, average='micro')
f1 = f1_score(ground_truth, all_recommendations, average='micro')

# Calculate relevance scores for NDCG
relevance_scores = {}
for user_id in test_data['User-ID'].unique():
    relevance_scores[user_id] = [1 if item in ground_truth[user_id] else 0 for item in all_recommendations[user_id]]

# Calculate NDCG
ndcg = ndcg_score(list(ground_truth.values()), list(relevance_scores.values()))

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("NDCG:", ndcg)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [90]:
def recommend(book_name):
    try:
        if book_name in pt.index:
            index = np.where(pt.index == book_name)[0][0]
            similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:5]
            
            data = []
            for i in similar_items:
                item = []
                temp_df = books[books['Book-Title'] == pt.index[i[0]]]
                item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
                item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
                item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
                
                data.append(item)
            
            return data
        else:
            print("Book not found in the dataset.")
            return []
    except Exception as e:
        print("Error in recommend function:", e)
        return []


In [91]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import ndcg_score

# Assuming similarity_scores and pt are already calculated
# Split data into train and test sets (you can use any method, e.g., train_test_split)
train_data = final_ratings.sample(frac=0.8, random_state=42)
test_data = final_ratings.drop(train_data.index)

# Train the recommendation model (not applicable here as it's similarity-based)
# If you have a model that needs training, do it here

# Generate recommendations for each user in the test set
all_recommendations = {}
for user_id in test_data['User-ID'].unique():
    recommendations = recommend(user_id)  # Assuming you have a function to generate recommendations for a user
    all_recommendations[user_id] = recommendations

# Prepare ground truth data for users in the test set
ground_truth = {}
for user_id in test_data['User-ID'].unique():
    ground_truth[user_id] = final_ratings[(final_ratings['User-ID'] == user_id) & (final_ratings['Book-Rating'] >= 7)]['Book-Title'].tolist()

# Calculate evaluation metrics
precision = precision_score(ground_truth, all_recommendations, average='micro')
recall = recall_score(ground_truth, all_recommendations, average='micro')
f1 = f1_score(ground_truth, all_recommendations, average='micro')

# Calculate relevance scores for NDCG
relevance_scores = {}
for user_id in test_data['User-ID'].unique():
    relevance_scores[user_id] = [1 if item in ground_truth[user_id] else 0 for item in all_recommendations[user_id]]

# Calculate NDCG
ndcg = ndcg_score(list(ground_truth.values()), list(relevance_scores.values()))

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("NDCG:", ndcg)


Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not

ValueError: Expected array-like (array or non-string sequence), got {7158: ['The Notebook', 'Roses Are Red (Alex Cross Novels)', 'SHIPPING NEWS', 'A Walk to Remember', 'The Dogs of Babel (Today Show Book Club #12)', 'By the Light of the Moon', "Where the Heart Is (Oprah's Book Club (Paperback))", 'The Notebook', 'The Guardian', 'Icy Sparks', 'The Rescue', 'Jewel', 'Chicken Soup for the Soul (Chicken Soup for the Soul)', 'From the Corner of His Eye', 'Black and Blue', "Suzanne's Diary for Nicholas", 'The Wedding', 'Intensity'], 32440: ['Artemis Fowl (Artemis Fowl, Book 1)', 'Jack &amp; Jill (Alex Cross Novels)', 'Girl with a Pearl Earring', 'Life of Pi', 'The Secret Life of Bees', 'Silent Witness', 'Middlesex: A Novel', 'All That Remains (Kay Scarpetta Mysteries (Paperback))', 'A Year in Provence', 'The Thorn Birds', "ANGELA'S ASHES", 'House of Sand and Fog', 'Echoes', 'Peace Like a River', 'The Phantom Tollbooth', "Lamb : The Gospel According to Biff, Christ's Childhood Pal"], 33974: [

In [95]:
def generate_user_based_recommendations(user_id, similarity_matrix, interactions, top_n=10):
    # Find similar users to the target user
    similar_users = np.argsort(similarity_matrix[user_id])[::-1]  # Sort in descending order of similarity
    
    # Exclude the target user from the list of similar users
    similar_users = similar_users[similar_users != user_id]
    
    # Get items interacted by similar users
    items_interacted_by_similar_users = interactions[similar_users].toarray()
    
    # Find items not interacted by the target user
    items_not_interacted = np.where(interactions[user_id] == 0)[0]
    
    # Calculate recommendation scores for items not interacted by the target user
    recommendation_scores = np.sum(items_interacted_by_similar_users, axis=0)
    
    # Sort recommendation scores in descending order
    recommended_item_indices = np.argsort(recommendation_scores)[::-1]
    
    # Filter out items already interacted by the user
    recommended_items = [item for item in recommended_item_indices if item in items_not_interacted][:top_n]
    
    return recommended_items

# Example usage:
user_id = 123  # Specify the target user ID
top_n = 10  # Number of recommendations to generate
recommendations = generate_user_based_recommendations(user_id, similarity_matrix, interactions, top_n)
print("Top recommendations for user", user_id, ":", recommendations)


NameError: name 'similarity_matrix' is not defined

In [96]:
# Train the recommendation model (skip this step if your model does not require explicit training)
# For collaborative filtering or matrix factorization, you may directly use similarity_scores obtained from your preprocessing step

# Generate recommendations for users in the test set
all_recommendations = {}
for user_id in test_data['User-ID'].unique():
    recommendations = recommend(user_id)  # Assuming you have a function to generate recommendations for a user
    all_recommendations[user_id] = recommendations

# Prepare ground truth data for users in the test set
ground_truth = {}
for user_id in test_data['User-ID'].unique():
    ground_truth[user_id] = interactions[(interactions['User-ID'] == user_id) & (interactions['Book-Rating'] >= 7)]['ISBN'].tolist()

# Calculate evaluation metrics
precision = precision_score(ground_truth, all_recommendations, average='micro')
recall = recall_score(ground_truth, all_recommendations, average='micro')
f1 = f1_score(ground_truth, all_recommendations, average='micro')

# Calculate relevance scores for NDCG
relevance_scores = {}
for user_id in test_data['User-ID'].unique():
    relevance_scores[user_id] = [1 if item in ground_truth[user_id] else 0 for item in all_recommendations[user_id]]

# Calculate NDCG
ndcg = ndcg_score(list(ground_truth.values()), list(relevance_scores.values()))

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("NDCG:", ndcg)


Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not found in the dataset.
Book not

NameError: name 'interactions' is not defined